In [ ]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [ ]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import *
from quantfreedom.nb_funcs.nb_simulate_base import nb_sim_backtest
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [ ]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=10500,
)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings(
    array_size=1000,
    gains_pct_filter=0,
    total_trade_filter=40,
    upside_filter=0,
)

# backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([2, 4, 6, 8]) * 3,
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 8, 10]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.2]),
    sl_based_on_lookback=np.array([20, 40, 60]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.05, 0.1, 0.5, 1, 2]),
    trail_sl_when_pct=np.array([0.5, 1, 3]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=100,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [ ]:
bt_strat_df = nb_sim_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_bool=False,
    long_short="long",
    nb_strat_evaluate=nb_strat_long_evaluate,
    nb_strat_get_current_ind_settings=nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str=nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings=nb_strat_get_total_ind_settings,
    nb_strat_ind_creator=nb_strat_bt_create_ind,
    static_os=static_os,
)

In [ ]:
bt_strat_df.sort_values(by=["qf_score"], ascending=False)